# AQUÍ CREAREMOS LAS TABLAS DE DIMENSION Provincia, Partido, Localidad, Trimestre, Año y Tecnología 

In [1]:
import pandas as pd
import unidecode


In [2]:
def normalizacion_palabras(palabra_corregir):
    '''Saca las mayusculas y las tildes de una palabrade las palabras y pone la primera letra de cada palabra en mayúscula 
    (exceptuando las las palabras que conectan otras palabras como la, el, de, etc).'''
    excepciones = ["la", "el", "de", "y", "del", "los", "las", "en", "al", "por", "con", "a", "para"]
    # Convertimos la palabra a minúsculas para empezar desde una base común
    palabras = palabra_corregir.lower().split()
    
    # Corregimos las palabras y quitamos las tildes
    palabras_corregidas = []
    for i, palabra in enumerate(palabras):
        # Si la palabra está en la lista de excepciones, no la capitalizamos (a menos que sea la primera)
        if palabra in excepciones and i != 0:
            palabras_corregidas.append(palabra)
        else:
            # Capitalizamos la palabra
            palabra_corregida = palabra.capitalize()
            # Quitamos las tildes
            palabra_corregida = unidecode.unidecode(palabra_corregida)
            palabras_corregidas.append(palabra_corregida)
    
    # Volver a juntar las palabras
    palabras_corregidas = ' '.join(palabras_corregidas)
    
    return palabras_corregidas

<u>TRIMENSTRE</u>

In [3]:
trimestre=pd.DataFrame({"Trimestre":[1,2,3,4],
                        "Meses":["Enero-Marzo","Abril-Junio","Julio-Septiembre","Octubre-Dicimbre"]})
trimestre

,Trimestre,Meses
0,1,Enero-Marzo
1,2,Abril-Junio
2,3,Julio-Septiembre
3,4,Octubre-Dicimbre


In [4]:
trimestre.to_csv('./Database/Tablas_de_dimension/Trimestres.csv',index=False)

<u>Provincias</u>

In [5]:
prov=["Caba","Buenos Aires", "Catamarca", "Chaco", "Chubut", "Cordoba", "Corrientes", 
            "Entre Rios", "Formosa", "Jujuy", "La Pampa", "La Rioja", "Mendoza", 
            "Misiones", "Neuquen", "Rio Negro", "Salta", "San Juan", "San Luis", 
            "Santa Cruz", "Santa Fe", "Santiago del Estero", "Tierra del Fuego", "Tucuman"]

In [6]:
provincias=pd.DataFrame({"Id_Provincia":[i for i in range(1,len(prov)+1)],
                         "Provincia":prov})
provincias

,Id_Provincia,Provincia
0,1,Caba
1,2,Buenos Aires
2,3,Catamarca
3,4,Chaco
4,5,Chubut
5,6,Cordoba
6,7,Corrientes
7,8,Entre Rios
8,9,Formosa
9,10,Jujuy


In [7]:
provincias.to_csv('./Database/Tablas_de_dimension/Provincias.csv',index=False)

<u>Partidos</u>

Creamos los dataframe y corregimos los nombres de las columnas de una o de ellos 

In [8]:
Acc_vel_loc_sinrangos = pd.read_excel("./Database/internet.xlsx", sheet_name="Acc_vel_loc_sinrangos")
Acc_vel_loc_sinrangos=Acc_vel_loc_sinrangos.rename(columns={"Partido":"provincia",
                                                 "Localidad":"partido",
                                                 "link Indec":"localidad",
                                                 "Velocidad (Mbps)":"link Indec",
                                                 "Provincia":"velocidad"})

Acc_vel_loc_sinrangos=Acc_vel_loc_sinrangos.rename(columns={"provincia":"Provincia",
                                                 "partido":"Partido",
                                                 "localidad":"Localidad",
                                                 "velocidad":"Velocidad (Mbps)"})
Accesos_tecnologia_localidad = pd.read_excel("./Database/internet.xlsx", sheet_name="Accesos_tecnologia_localidad")

Sacamos solo las columnas que nos interesan 

In [9]:
df1 = Acc_vel_loc_sinrangos[["Provincia","Partido","Localidad"]]
df2 = Accesos_tecnologia_localidad[["Provincia","Partido","Localidad"]]

Concatenamos ambos verticalmente y creamos el dataset partido sacando la localidad

In [10]:
df = pd.concat([df1, df2], ignore_index=True)
partido = df.drop(columns="Localidad")

Eliminamos los duplicados y los NaN 

In [11]:
partido = partido.drop_duplicates(subset=["Partido"])
partido1 = partido.dropna()

Corregimos algunos valores en formato incorrecto

In [12]:
partido1["Partido"] = partido1["Partido"].replace("Ciudad Autónoma de Buenos Aires", "Caba")
partido2 = partido1[partido1['Partido'] != 'ANTARTIDA ARGENTINA']
partido3 = partido2[partido2['Partido'] != 'OTROS']

Normalizamos el formato de las palabras 

In [13]:
partido3.loc[:, "Partido"]=partido3["Partido"].apply(normalizacion_palabras)
partido3.loc[:, "Provincia"]=partido3["Provincia"].apply(normalizacion_palabras)

Traemos el Id_Provincia y eliminamos la columna Provincia

In [14]:
partido3=partido3.merge(provincias[["Provincia", "Id_Provincia"]], left_on="Provincia", right_on="Provincia", how="left")
partido4=partido3.drop(columns="Provincia")
partido4.head()

,Partido,Id_Provincia
0,25 de Mayo,2
1,9 de Julio,2
2,Adolfo Alsina,2
3,Adolfo Gonzales Chaves,2
4,Alberti,2


Corroboramos que no hay duplicados una vez normalizado el formato del texto de Partido 

In [15]:
partido5 = partido4.drop_duplicates(subset=["Partido"],ignore_index=True)

Creamos el Id_Partido

In [16]:
partido5["Id_Partido"]=[i for i in range(1,partido5.shape[0]+1)]

C:\Users\maria\AppData\Local\Temp\ipykernel_13496\685838667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partido5["Id_Partido"]=[i for i in range(1,partido5.shape[0]+1)]


In [17]:
partido5

,Partido,Id_Provincia,Id_Partido
0,25 de Mayo,2,1
1,9 de Julio,2,2
2,Adolfo Alsina,2,3
3,Adolfo Gonzales Chaves,2,4
4,Alberti,2,5
...,...,...,...
427,Simoca,24,428
428,Tafi del Valle,24,429
429,Tafi Viejo,24,430
430,Trancas,24,431


In [18]:
partido5.to_csv('./Database/Tablas_de_dimension/Partidos.csv',index=False)

<u>Localidades</u>

Eliminamos los duplicados de la columna Localidades y los NaN

In [19]:
localidad = df.drop_duplicates(subset=["Localidad"])
localidad = localidad.dropna()

Hacemos las mismas correcciones que antes. Primero normalizamos el formato de las columnas Provincia y Partido

In [20]:
localidad.loc[:, "Partido"]=df["Partido"].apply(normalizacion_palabras)
localidad.loc[:, "Provincia"]=df["Provincia"].apply(normalizacion_palabras)
localidad.loc[:, "Localidad"]=localidad["Localidad"].apply(normalizacion_palabras)
localidad["Localidad"] = localidad["Localidad"].replace("Ciudad Autonoma de Buenos Aires", "Caba")
localidad["Partido"] = localidad["Partido"].replace("Ciudad Autonoma de Buenos Aires", "Caba")

In [21]:
localidad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2802 entries, 0 to 24908
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Provincia  2802 non-null   object
 1   Partido    2802 non-null   object
 2   Localidad  2802 non-null   object
dtypes: object(3)
memory usage: 87.6+ KB


In [22]:
localidad = localidad.drop_duplicates(subset=["Localidad"],ignore_index=True)
localidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Provincia  2793 non-null   object
 1   Partido    2793 non-null   object
 2   Localidad  2793 non-null   object
dtypes: object(3)
memory usage: 65.6+ KB


Traemos los id de ambos y eliminamos las columnas

In [23]:
localidad = localidad.merge(provincias[["Provincia", "Id_Provincia"]], left_on="Provincia", right_on="Provincia", how="left")
localidad = localidad.merge(partido5[["Partido", "Id_Partido"]], left_on="Partido", right_on="Partido", how="left")
localidad1 = localidad.drop(columns=["Provincia","Partido"])

In [24]:
localidad1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Localidad     2793 non-null   object
 1   Id_Provincia  2793 non-null   int64 
 2   Id_Partido    2793 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 65.6+ KB


Creamos la columna Id_Localidad

In [25]:
localidad1["Id_Localidad"]=[i for i in range(1,localidad1.shape[0]+1)]

In [26]:
localidad1

,Localidad,Id_Provincia,Id_Partido,Id_Localidad
0,25 de Mayo,2,1,1
1,Agustin Mosconi,2,1,2
2,Del Valle,2,1,3
3,Ernestina,2,1,4
4,Gobernador Ugarte,2,1,5
...,...,...,...,...
2788,Castro Barros,12,48,2789
2789,Suriyaco,12,297,2790
2790,Tuyubil,12,297,2791
2791,Jague,12,299,2792


In [58]:
localidad1.to_csv('./Database/Tablas_de_dimension/Localidades.csv',index=False)

<u>Años</u>

In [31]:
anios={"Año":[i for i in range(2014,2025)]}

In [32]:
anios_df=pd.DataFrame(anios)

In [33]:
anios_df

,Año
0,2014
1,2015
2,2016
3,2017
4,2018
5,2019
6,2020
7,2021
8,2022
9,2023


In [34]:
anios_df.to_csv('./Database/Tablas_de_dimension/Años.csv',index=False)

<u>Tecnologías</u>

In [47]:
Accesos_tecnologia_localidad=pd.read_excel("./Database/internet.xlsx", sheet_name="Accesos_tecnologia_localidad")

Creamos el dataframe

In [52]:
tecnologia=pd.DataFrame(Accesos_tecnologia_localidad["Tecnologia"])
tecnologia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7753 entries, 0 to 7752
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tecnologia  7753 non-null   object
dtypes: object(1)
memory usage: 60.7+ KB


Normalizamos el formato del texto

In [53]:
tecnologia.loc[:, "Tecnologia"]=tecnologia["Tecnologia"].apply(normalizacion_palabras)

Eliminamos los dulicados

In [54]:
tecnologia = tecnologia.drop_duplicates(ignore_index=True)
tecnologia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tecnologia  8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes


Creamos el Id_Tecnologia

In [55]:
tecnologia["Id_Tecnologia"]=[i for i in range(1,tecnologia.shape[0]+1)]

In [56]:
tecnologia

,Tecnologia,Id_Tecnologia
0,Adsl,1
1,Cablemodem,2
2,Fibra Optica,3
3,Satelital,4
4,Wireless,5
5,Otros,6
6,Dial Up,7
7,Wimax,8


In [57]:
tecnologia.to_csv('./Database/Tablas_de_dimension/Tecnologias.csv',index=False)